# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '3D', 'insta_score': 0.365, 'expected_score': -80, 'adjustment': 18},
 {'call': 'PASS',
  'insta_score': 0.266,
  'expected_score': -81,
  'adjustment': 13},
 {'call': 'X', 'insta_score': 0.123, 'expected_score': -140, 'adjustment': 6},
 {'call': '4D', 'insta_score': 0.128, 'expected_score': -178, 'adjustment': 6}]

In [4]:
bid.samples

['J.J8x.A98xxx.Txx A9x.Kxxxx.Tx.Kxx K8.AQT.Jxx.AJ8xx QTxxxxx.9x.KQ.Q9 0.43463 \n 1H-1N-2S-3D-P-P-4S-P-P-P (7.6)  \n 1H-1N-2S-P-P-3C-3S-4D-P-P-P (10.5)  \n 1H-1N-2S-4D-P-P-P (10.5)  \n 1H-1N-2S-X-P-3C-X-P-P-P (9.4) ',
 'J.J8x.A98xxx.Txx K.AQT9xx.JT.J9xx Q9xx.Kx.KQx.KQ8x AT8xxxx.xx.xx.Ax 0.39905 \n 1H-1N-2S-3D-3H-P-P-4D-P-P-P (9.3)  \n 1H-1N-2S-P-3H-P-P-4D-P-P-P (9.3)  \n 1H-1N-2S-4D-P-P-P (9.3)  \n 1H-1N-2S-X-3H-P-P-4D-X-P-P-P (9.3) ',
 'J.J8x.A98xxx.Txx T8x.AKxxxx.T.QJx Kx.QT9x.QJxx.AK9 AQ9xxxx..Kx.8xxx 0.38197 \n 1H-1N-2S-3D-3H-P-4S-P-P-P (10.0)  \n 1H-1N-2S-P-3D-P-4S-P-P-P (10.0)  \n 1H-1N-2S-4D-P-4H-X-P-P-P (6.1)  \n 1H-1N-2S-X-P-P-P (10.0) ',
 'J.J8x.A98xxx.Txx 98.AKTxxx.Tx.QJx AKx.Q9x.KQJx.xxx QTxxxxx.x.x.AK98 0.35863 \n 1H-1N-2S-3D-3H-P-4S-P-P-P (10.0)  \n 1H-1N-2S-P-3H-P-4S-P-P-P (10.0)  \n 1H-1N-2S-4D-P-P-P (7.7)  \n 1H-1N-2S-X-3H-P-4S-P-P-P (10.0) ',
 'J.J8x.A98xxx.Txx A.KQTxxx.Jx.J9xx KT8x.A9x.KQx.A8x Q9xxxxx.x.Tx.KQx 0.35815 \n 1H-1N-2S-3D-P-P-3S-4D-P-P-P (9.1)  \n 1H-1N-2S-

In [5]:
auction = ["PAD_START", "PAD_START", "1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.554,
  'expected_score': -249,
  'adjustment': 28},
 {'call': 'X', 'insta_score': 0.404, 'expected_score': -333, 'adjustment': 20}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [6]:
bid.samples

['QT9xxxx..KT.Q8xx J.J8x.A98xxx.Txx Axx.ATxxx.xx.KJx K8.KQ9xx.QJx.A9x 0.30275 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (10.2)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (10.2) ',
 'AQ9xxx.x.Q.J8xxx J.J8x.A98xxx.Txx Txx.KQ9xx.x.KQ9x K8x.ATxx.KJTxx.A 0.25517 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (9.2)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (9.2) ',
 'AT9xxx.x.KJTx.8x J.J8x.A98xxx.Txx K8xx.KT9xx.x.Axx Qx.AQxx.Qx.KQJ9x 0.24481 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (9.9)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (9.9) ',
 'AT9xxx.x.x.AJ9xx J.J8x.A98xxx.Txx KQ8x.KQTxxx.x.8x xx.A9x.KQJTx.KQx 0.24111 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (11.4)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (11.4) ',
 'KTxxxx.x.KQxx.xx J.J8x.A98xxx.Txx Qxx.KQTxx.x.KJxx A98.A9xx.JT.AQ98 0.23162 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (7.7)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (7.7) ',
 'AKT8xxx.x.xxx.xx J.J8x.A98xxx.Txx xxx.Q9xxxx..AKQ9 Q9.AKT.KQJT.J8xx 0.21738 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (10.6)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (10.6) ',
 'Qxxxxx.A.Qx.J8xx J.J8x.A98xxx.Txx KTx.QTxxx.x.KQ9x A98.K9xx.KJTx.Ax 

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

J.J8x.A98xxx.Txx Q8xx.xxx.KQ.A8xx AKT9xx.AKQx.JT.K xx.T9x.xxx.QJ9xx
J.J8x.A98xxx.Txx x.9x.JTx.AJ98xxx AKQTxx.AKxx.xx.K 98xxx.QTxx.KQ.Qx
J.J8x.A98xxx.Txx 8xxx.9.Txx.AJ9xx AQT9xx.AKTx.KJ.Q Kx.Qxxxx.Qx.K8xx
J.J8x.A98xxx.Txx 98x.T9x.JTx.AQxx AKQTxxx.AKQx.x.J xx.xxx.KQx.K98xx
J.J8x.A98xxx.Txx 8xx.Q9x.QJx.J8xx AKQxxxx.AKTx.K.x T9.xxx.Txx.AKQ9x
J.J8x.A98xxx.Txx QTx.T9xxx.xx.KQJ AK98xx.AKQx.K.xx xxx.x.QJTx.A98xx
J.J8x.A98xxx.Txx T.QTxx.Txx.KQ8xx AQxxxxx.AK9x.K.A K98x.xx.QJx.J9xx
J.J8x.A98xxx.Txx KTxx.Qxx.JTxx.9x AQ98xx.AK9x..AKx xx.Txx.KQx.QJ8xx
J.J8x.A98xxx.Txx 9xx.Txx.KJxx.AKx AKQ8xx.AKQx..Q98 Txx.9xx.QTx.Jxxx
J.J8x.A98xxx.Txx K9xx.T9x.Qx.Jxxx AQTxxxx.AKxx.K.K 8.Qxx.JTxx.AQ98x
